In [96]:
import pandas as pd
import numpy as np

In [97]:
n = 578

In [98]:
rows = pd.read_csv('probability_matrix.csv', index_col='Unnamed: 0').to_numpy()
rows

array([[0.        , 0.00466516, 0.00759291, ..., 0.00469103, 0.0047618 ,
        0.00346845],
       [0.00307812, 0.        , 0.00577419, ..., 0.0035649 , 0.00361874,
        0.00263503],
       [0.00165602, 0.00190773, 0.        , ..., 0.00191834, 0.00194736,
        0.00141735],
       ...,
       [0.00290714, 0.00334839, 0.00545428, ..., 0.        , 0.00341784,
        0.00248861],
       [0.00199286, 0.00229565, 0.00374194, ..., 0.00230841, 0.        ,
        0.00170573],
       [0.04810323, 0.05502574, 0.08680229, ..., 0.05531536, 0.05610671,
        0.        ]])

In [114]:
from collections import deque
from tqdm import tqdm

In [115]:
fans = np.random.rand(578)*10000
fans = fans.astype(int)

In [118]:
round = []


for i in tqdm(range(n)):

    fan = 0
    for _ in range(10):
        activated = np.array([False for _ in range(n)])
        activated[i] = True
        new_ones = deque([i])
        
        while new_ones:
            j = new_ones.popleft()
            fan += fans[j]
            row = rows[j]
            p = np.random.uniform(0, 1, n)
            success = p < row
            success = success & ~activated
            new_ones.extend(np.where(success)[0].tolist())
            activated |= success
    
    fan//=10
    round.append(fan)


100%|██████████| 578/578 [00:34<00:00, 16.53it/s]
